In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import missingno as msno
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import warnings
warnings.simplefilter('ignore') # 
from pandarallel import pandarallel
 
# Initialization
pandarallel.initialize()

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
fund_sparsity =  pd.read_csv('2024_04_11_fund_sparisty.csv')

#### Remove MSCI EAFE NR USD for the passive measures and remove top 5 percentile for other measures

In [3]:
fund_sparsity.loc[fund_sparsity['prim_prospectus_bm'] == 'MSCI EAFE NR USD', ['percent_benchmark_median_passive', 'percent_benchmark_avg_passive', 'percent_benchmark_largest_passive']] = np.nan

In [4]:
# Assuming fund_sparsity is your DataFrame
columns_to_process = [
    'percent_benchmark_3', 'percent_benchmark_7', 'percent_benchmark_11',
    'percent_benchmark_15', 'percent_benchmark_19', 'percent_benchmark_median_passive',
    'percent_benchmark_avg_passive', 'percent_benchmark_largest_passive', 'percent_benchmark_0',
    'percent_within_3', 'percent_within_7', 'percent_within_11',
    'percent_within_15', 'percent_within_19'
]

def replace_top_5_percentile(group):
    # For each column to process, replace top 5 percentile values with NaN
    for column in columns_to_process:
        # Calculate the 95th percentile value for the current column
        percentile_95 = group[column].quantile(0.95)
        # Replace values greater than the 95th percentile with NaN
        group.loc[group[column] > percentile_95, column] = np.nan
    return group

# Group the DataFrame by 'quarter' and apply the replacement function
fund_sparsity = fund_sparsity.groupby('quarter').apply(replace_top_5_percentile)
# Now fund_sparsity_updated contains the updated DataFrame
fund_sparsity = fund_sparsity.drop(columns=['quarter']).reset_index().drop(columns=['level_1'])

In [5]:
fund_sparsity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425646 entries, 0 to 425645
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   quarter                            425646 non-null  object 
 1   wficn                              425646 non-null  float64
 2   prim_prospectus_bm                 145457 non-null  object 
 3   percent_within_3                   424581 non-null  float64
 4   if_past_quarter_missing_3          425646 non-null  int64  
 5   percent_within_7                   417914 non-null  float64
 6   if_past_quarter_missing_7          425646 non-null  int64  
 7   percent_within_11                  415561 non-null  float64
 8   if_past_quarter_missing_11         425646 non-null  int64  
 9   percent_within_15                  414317 non-null  float64
 10  if_past_quarter_missing_15         425646 non-null  int64  
 11  percent_within_19                  4140

In [6]:
mutual_fund_holding = pd.read_csv('mutual_fund_holding_noDrops.csv')

In [7]:
mutual_fund_holding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53285149 entries, 0 to 53285148
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   wficn               float64
 1   permno              int64  
 2   prim_prospectus_bm  object 
 3   quarter             object 
 4   dollar_holdings     float64
 5   shares              float64
 6   prc                 float64
dtypes: float64(4), int64(1), object(2)
memory usage: 2.8+ GB


In [8]:
# Step 2: Merge fund_sparsity_prepared with mutual_fund_holding
mutual_fund_holding = pd.merge(mutual_fund_holding, fund_sparsity, on=['wficn', 'quarter'], how='left',suffixes=('','_fund_agg'))

In [9]:
mutual_fund_holding

,wficn,permno,prim_prospectus_bm,quarter,dollar_holdings,shares,prc,prim_prospectus_bm_fund_agg,percent_within_3,if_past_quarter_missing_3,percent_within_7,if_past_quarter_missing_7,percent_within_11,if_past_quarter_missing_11,percent_within_15,if_past_quarter_missing_15,percent_within_19,if_past_quarter_missing_19,percent_benchmark_0,percent_benchmark_3,percent_benchmark_7,percent_benchmark_11,percent_benchmark_15,percent_benchmark_19,percent_benchmark_median_passive,percent_benchmark_avg_passive,percent_benchmark_largest_passive
0,103544.00,78736,NaN,1996Q3,6125.00,2800.00,2.19,NaN,0.98,0,0.98,0,0.97,0,0.97,0,0.96,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101068.00,10015,NaN,1983Q4,300000.00,40000.00,7.50,NaN,0.84,0,0.77,0,0.73,0,0.72,1,0.72,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101068.00,10015,NaN,1984Q1,262500.00,40000.00,6.56,NaN,0.74,0,0.68,0,0.63,0,0.63,1,0.62,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101068.00,10015,NaN,1984Q2,237500.00,40000.00,5.94,NaN,0.64,0,0.59,0,0.56,0,0.55,1,0.54,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101080.00,10015,NaN,1986Q1,490000.00,40000.00,12.25,NaN,1.00,1,1.00,1,1.00,1,1.00,1,1.00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53285144,603675.00,15857,NaN,2020Q3,8490.00,1000.00,8.49,NaN,0.97,0,0.95,0,0.94,0,0.94,0,0.94,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53285145,603676.00,15857,NaN,2020Q3,16504.56,1944.00,8.49,NaN,0.94,0,0.91,0,0.89,0,0.88,0,0.88,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53285146,603798.00,15857,NaN,2020Q3,30648.90,3610.00,8.49,NaN,0.95,0,0.92,0,0.90,0,0.89,0,0.88,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53285147,604248.00,15857,NaN,2020Q3,494457.59,58240.00,8.49,NaN,0.83,0,0.64,0,0.57,0,0.51,0,0.51,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1. Compute the equal weighted average

In [10]:
h_values = [3, 7, 11, 15, 19]
benchmark_means_dfs = []  # List to hold the benchmark dataframes with means
within_means_dfs = []     # List to hold the within dataframes with means

for h in h_values:
    # Filter the dataframe for the current h where there are no missing quarters
    # No filter needed
    filtered_df = mutual_fund_holding[['permno', 'quarter',f'percent_within_{h}',f'percent_benchmark_{h}']]
    
    # Further filter for non-NaN percent_benchmark values before calculating the mean for benchmark
    filtered_df_bm = filtered_df.dropna(subset=[f'percent_benchmark_{h}'])
    benchmark_means = filtered_df_bm.groupby(['permno', 'quarter'])[[f'percent_benchmark_{h}']].mean().reset_index()
    benchmark_means.rename(columns={f'percent_benchmark_{h}': f'percent_benchmark_{h}_equ_w_mean'}, inplace=True)
    benchmark_means_dfs.append(benchmark_means)
    
    # Calculate the mean for percent_within without additional NaN filtering
    within_means = filtered_df.groupby(['permno', 'quarter'])[[f'percent_within_{h}']].mean().reset_index()
    within_means.rename(columns={f'percent_within_{h}': f'percent_within_{h}_equ_w_mean'}, inplace=True)
    within_means_dfs.append(within_means)

# Assuming you want to merge these into comprehensive dataframes for benchmark and within separately
# Initialize the merged dataframes for benchmark and within metrics
equal_weighted_benchmark = benchmark_means_dfs[0]
for df in benchmark_means_dfs[1:]:
    equal_weighted_benchmark = pd.merge(equal_weighted_benchmark, df, on=['permno', 'quarter'], how='outer')

equal_weighted_within = within_means_dfs[0]
for df in within_means_dfs[1:]:
    equal_weighted_within = pd.merge(equal_weighted_within, df, on=['permno', 'quarter'], how='outer')

# If needed, combine these separate metrics back into a single DataFrame
equal_weighted_average = pd.merge(equal_weighted_benchmark, equal_weighted_within, on=['permno', 'quarter'], how='outer')

In [11]:
equal_weighted_average

,permno,quarter,percent_benchmark_3_equ_w_mean,percent_benchmark_7_equ_w_mean,percent_benchmark_11_equ_w_mean,percent_benchmark_15_equ_w_mean,percent_benchmark_19_equ_w_mean,percent_within_3_equ_w_mean,percent_within_7_equ_w_mean,percent_within_11_equ_w_mean,percent_within_15_equ_w_mean,percent_within_19_equ_w_mean
0,10001,1993Q4,NaN,NaN,NaN,NaN,NaN,0.98,0.98,0.98,0.98,0.98
1,10001,1994Q1,NaN,NaN,NaN,NaN,NaN,0.96,0.94,0.94,0.94,0.94
2,10001,1994Q2,NaN,NaN,NaN,NaN,NaN,0.96,0.95,0.95,0.95,0.95
3,10001,1994Q3,NaN,NaN,NaN,NaN,NaN,0.97,0.95,0.95,0.95,0.95
4,10001,1994Q4,NaN,NaN,NaN,NaN,NaN,0.99,0.98,0.98,0.98,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...
812341,93436,2019Q3,0.06,0.06,0.05,0.05,0.05,0.88,0.79,0.73,0.68,0.66
812342,93436,2019Q4,0.05,0.05,0.04,0.04,0.04,0.58,0.51,0.47,0.44,0.43
812343,93436,2020Q1,0.07,0.06,0.06,0.05,0.05,0.81,0.70,0.64,0.59,0.57
812344,93436,2020Q2,0.07,0.06,0.06,0.06,0.05,0.79,0.69,0.63,0.60,0.57


In [12]:
passive_measures = [
    "percent_benchmark_median_passive",
    "percent_benchmark_avg_passive",
    "percent_benchmark_largest_passive",
    "percent_benchmark_0"
]

In [13]:
passive_grouped_df = mutual_fund_holding.groupby(['permno', 'quarter'])[passive_measures].mean().reset_index()

In [14]:
equal_weighted_average = pd.merge(equal_weighted_average, passive_grouped_df, on=['permno', 'quarter'], how='outer')

In [15]:
# Assuming you have a dataframe named 'equal_weighted_average'
equal_weighted_average = equal_weighted_average.rename(columns={
    'percent_benchmark_median_passive': 'percent_benchmark_median_passive_equ_w_mean',
    'percent_benchmark_avg_passive': 'percent_benchmark_avg_passive_equ_w_mean',
    'percent_benchmark_largest_passive': 'percent_benchmark_largest_passive_equ_w_mean',
    'percent_benchmark_0': 'percent_benchmark_0_equ_w_mean'
})

In [16]:
equal_weighted_average

,permno,quarter,percent_benchmark_3_equ_w_mean,percent_benchmark_7_equ_w_mean,percent_benchmark_11_equ_w_mean,percent_benchmark_15_equ_w_mean,percent_benchmark_19_equ_w_mean,percent_within_3_equ_w_mean,percent_within_7_equ_w_mean,percent_within_11_equ_w_mean,percent_within_15_equ_w_mean,percent_within_19_equ_w_mean,percent_benchmark_median_passive_equ_w_mean,percent_benchmark_avg_passive_equ_w_mean,percent_benchmark_largest_passive_equ_w_mean,percent_benchmark_0_equ_w_mean
0,10001,1993Q4,NaN,NaN,NaN,NaN,NaN,0.98,0.98,0.98,0.98,0.98,NaN,NaN,NaN,NaN
1,10001,1994Q1,NaN,NaN,NaN,NaN,NaN,0.96,0.94,0.94,0.94,0.94,NaN,NaN,NaN,NaN
2,10001,1994Q2,NaN,NaN,NaN,NaN,NaN,0.96,0.95,0.95,0.95,0.95,NaN,NaN,NaN,NaN
3,10001,1994Q3,NaN,NaN,NaN,NaN,NaN,0.97,0.95,0.95,0.95,0.95,NaN,NaN,NaN,NaN
4,10001,1994Q4,NaN,NaN,NaN,NaN,NaN,0.99,0.98,0.98,0.98,0.98,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812341,93436,2019Q3,0.06,0.06,0.05,0.05,0.05,0.88,0.79,0.73,0.68,0.66,0.32,0.27,0.28,0.07
812342,93436,2019Q4,0.05,0.05,0.04,0.04,0.04,0.58,0.51,0.47,0.44,0.43,0.39,0.33,0.40,0.06
812343,93436,2020Q1,0.07,0.06,0.06,0.05,0.05,0.81,0.70,0.64,0.59,0.57,0.27,0.31,0.25,0.08
812344,93436,2020Q2,0.07,0.06,0.06,0.06,0.05,0.79,0.69,0.63,0.60,0.57,0.31,0.30,0.29,0.08


## 2.Ownership_weighted

In [17]:
# Calculate the total number of shares of each stock (permno) owned by all funds in each quarter
#total_shares_by_stock_quarter = mutual_fund_holding.groupby(['permno', 'quarter'])['shares'].transform('sum')

# Calculate the weight of each fund's ownership of a stock
#mutual_fund_holding['ownership_weight'] = mutual_fund_holding['shares'] / total_shares_by_stock_quarter

In [18]:
h_values = [3, 7, 11, 15, 19]
benchmark_dfs = []  # List to hold the benchmark dataframes
within_dfs = []     # List to hold the within dataframes

for h in h_values:
    # Do not Filter the dataframe for the current h where there is no missing quarters
    filtered_df = mutual_fund_holding[['permno', 'quarter',f'percent_within_{h}',f'percent_benchmark_{h}','shares']]
    filtered_df_bm = filtered_df[filtered_df[f'percent_benchmark_{h}'].notna()]
    
    total_shares_by_stock_quarter = filtered_df.groupby(['permno', 'quarter'])['shares'].transform('sum')
    filtered_df['ownership_weight'] = filtered_df['shares'] / total_shares_by_stock_quarter

    total_shares_by_stock_quarter_bm = filtered_df_bm.groupby(['permno', 'quarter'])['shares'].transform('sum')
    filtered_df_bm['ownership_weight'] = filtered_df_bm['shares'] / total_shares_by_stock_quarter_bm
    
    # Calculate the weighted metrics separately
    filtered_df_bm[f'weighted_percent_benchmark_{h}'] = filtered_df_bm['ownership_weight'] * filtered_df_bm[f'percent_benchmark_{h}']
    filtered_df[f'weighted_percent_within_{h}'] = filtered_df['ownership_weight'] * filtered_df[f'percent_within_{h}']

    # Separate grouping and summing for weighted_percent_benchmark
    benchmark_grouped_df = filtered_df_bm.groupby(['permno', 'quarter'])[[f'weighted_percent_benchmark_{h}']].sum().reset_index()
    benchmark_grouped_df.rename(columns={f'weighted_percent_benchmark_{h}': f'percent_benchmark_{h}_ow_weighted'}, inplace=True)
    benchmark_dfs.append(benchmark_grouped_df)

    # Separate grouping and summing for weighted_percent_within
    within_grouped_df = filtered_df.groupby(['permno', 'quarter'])[[f'weighted_percent_within_{h}']].sum().reset_index()
    within_grouped_df.rename(columns={f'weighted_percent_within_{h}': f'percent_within_{h}_ow_weighted'}, inplace=True)
    within_dfs.append(within_grouped_df)

# At this point, benchmark_dfs and within_dfs contain separate DataFrames for each metric across different h values

# To merge these separate lists of DataFrames into comprehensive ones for benchmark and within metrics:
# Initialize with the first dataframe as the base for merging for each metric
ownership_share_weighted_benchmark = benchmark_dfs[0].copy()
ownership_share_weighted_within = within_dfs[0].copy()

# Merge the benchmark dataframes
for df in benchmark_dfs[1:]:
    ownership_share_weighted_benchmark = pd.merge(ownership_share_weighted_benchmark, df, on=['permno', 'quarter'], how='outer')

# Merge the within dataframes
for df in within_dfs[1:]:
    ownership_share_weighted_within = pd.merge(ownership_share_weighted_within, df, on=['permno', 'quarter'], how='outer')

# If you need to combine these separate metrics back into a single DataFrame, merge them
ownership_share_weighted_average = pd.merge(ownership_share_weighted_benchmark, ownership_share_weighted_within, on=['permno', 'quarter'], how='outer')

In [19]:
ownership_share_weighted_average.describe().round(4)

,permno,percent_benchmark_3_ow_weighted,percent_benchmark_7_ow_weighted,percent_benchmark_11_ow_weighted,percent_benchmark_15_ow_weighted,percent_benchmark_19_ow_weighted,percent_within_3_ow_weighted,percent_within_7_ow_weighted,percent_within_11_ow_weighted,percent_within_15_ow_weighted,percent_within_19_ow_weighted
count,812346.00,368485.00,367602.00,367277.00,366739.00,366878.00,812346.00,812346.00,812346.00,812346.00,812346.00
mean,61742.23,0.06,0.06,0.05,0.05,0.05,0.81,0.70,0.64,0.60,0.57
std,27777.95,0.07,0.06,0.06,0.06,0.06,0.11,0.15,0.17,0.18,0.18
min,10001.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,38295.00,0.03,0.03,0.03,0.03,0.03,0.75,0.61,0.53,0.48,0.46
50%,76023.00,0.05,0.05,0.04,0.04,0.04,0.82,0.70,0.63,0.59,0.56
75%,84345.00,0.07,0.06,0.06,0.06,0.06,0.89,0.80,0.75,0.71,0.69
max,93436.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [20]:
passive_measures = [
    "percent_benchmark_median_passive",
    "percent_benchmark_avg_passive",
    "percent_benchmark_largest_passive",
    "percent_benchmark_0"
]
# List to store grouped DataFrames for each measure
grouped_dfs = []

for measure in passive_measures:
    # Filter rows where the current measure is not null
    filtered_df = mutual_fund_holding[['permno', 'quarter',measure,'shares']]
    filtered_df = filtered_df[filtered_df[measure].notna()]

    # Calculate the total number of shares of each stock (permno) owned by all funds in each quarter for the filtered DataFrame
    total_shares_by_stock_quarter = filtered_df.groupby(['permno', 'quarter'])['shares'].transform('sum')

    # Calculate the weight of each fund's ownership of a stock in the filtered DataFrame
    filtered_df['ownership_weight'] = filtered_df['shares'] / total_shares_by_stock_quarter

    # Calculate the weighted measure for the filtered DataFrame
    filtered_df[f'weighted_{measure}'] = filtered_df['ownership_weight'] * filtered_df[measure]

    # Group by permno and quarter, sum the weighted measures, and reset index
    grouped_df = filtered_df.groupby(['permno', 'quarter'])[[f'weighted_{measure}']].sum().reset_index()
    grouped_df.rename(columns={f'weighted_{measure}': f'{measure}_ow_weighted'}, inplace=True)
    grouped_dfs.append(grouped_df)

# If you need to merge these separate DataFrames into one:
# Initialize with the first DataFrame as the base for merging for each measure
if grouped_dfs:
    passive_ow_grouped_df = grouped_dfs[0]
    for df in grouped_dfs[1:]:
        passive_ow_grouped_df = pd.merge(passive_ow_grouped_df, df, on=['permno', 'quarter'], how='outer')

In [21]:
passive_ow_grouped_df

,permno,quarter,percent_benchmark_median_passive_ow_weighted,percent_benchmark_avg_passive_ow_weighted,percent_benchmark_largest_passive_ow_weighted,percent_benchmark_0_ow_weighted
0,10001,2000Q2,0.21,0.21,0.21,0.04
1,10001,2004Q2,0.42,0.45,0.40,NaN
2,10001,2004Q4,0.74,0.80,0.72,NaN
3,10001,2006Q1,0.92,0.92,0.92,NaN
4,10001,2006Q2,0.92,0.92,0.94,NaN
...,...,...,...,...,...,...
388553,93436,2019Q3,0.33,0.30,0.31,0.06
388554,93436,2019Q4,0.37,0.33,0.38,0.06
388555,93436,2020Q1,0.24,0.27,0.23,0.06
388556,93436,2020Q2,0.37,0.36,0.35,0.07


In [22]:
ownership_share_weighted_average = pd.merge(ownership_share_weighted_average, passive_ow_grouped_df, on=['permno', 'quarter'], how='outer')

In [23]:
# Now merged_df contains the combined data from both equal_weighted_average and ownership_share_weighted_average DataFrames
two_weighted_average = pd.merge(equal_weighted_average, ownership_share_weighted_average, on=['permno', 'quarter'], how='outer')

In [24]:
two_weighted_average.describe().round(2)

,permno,percent_benchmark_3_equ_w_mean,percent_benchmark_7_equ_w_mean,percent_benchmark_11_equ_w_mean,percent_benchmark_15_equ_w_mean,percent_benchmark_19_equ_w_mean,percent_within_3_equ_w_mean,percent_within_7_equ_w_mean,percent_within_11_equ_w_mean,percent_within_15_equ_w_mean,percent_within_19_equ_w_mean,percent_benchmark_median_passive_equ_w_mean,percent_benchmark_avg_passive_equ_w_mean,percent_benchmark_largest_passive_equ_w_mean,percent_benchmark_0_equ_w_mean,percent_benchmark_3_ow_weighted,percent_benchmark_7_ow_weighted,percent_benchmark_11_ow_weighted,percent_benchmark_15_ow_weighted,percent_benchmark_19_ow_weighted,percent_within_3_ow_weighted,percent_within_7_ow_weighted,percent_within_11_ow_weighted,percent_within_15_ow_weighted,percent_within_19_ow_weighted,percent_benchmark_median_passive_ow_weighted,percent_benchmark_avg_passive_ow_weighted,percent_benchmark_largest_passive_ow_weighted,percent_benchmark_0_ow_weighted
count,812346.00,368485.00,367602.00,367277.00,366739.00,366878.00,811361.00,807841.00,805660.00,805117.00,804979.00,362216.00,360859.00,360788.00,368416.00,368485.00,367602.00,367277.00,366739.00,366878.00,812346.00,812346.00,812346.00,812346.00,812346.00,362216.00,360859.00,360788.00,368416.00
mean,61742.23,0.07,0.06,0.06,0.06,0.06,0.83,0.75,0.70,0.67,0.66,0.45,0.42,0.44,0.08,0.06,0.06,0.05,0.05,0.05,0.81,0.70,0.64,0.60,0.57,0.34,0.33,0.34,0.07
std,27777.95,0.07,0.06,0.06,0.06,0.06,0.09,0.12,0.13,0.14,0.14,0.22,0.22,0.23,0.08,0.07,0.06,0.06,0.06,0.06,0.11,0.15,0.17,0.18,0.18,0.23,0.22,0.23,0.08
min,10001.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,38295.00,0.04,0.04,0.04,0.04,0.04,0.78,0.67,0.61,0.58,0.56,0.28,0.27,0.28,0.05,0.03,0.03,0.03,0.03,0.03,0.75,0.61,0.53,0.48,0.46,0.18,0.17,0.17,0.04
50%,76023.00,0.06,0.05,0.05,0.05,0.05,0.84,0.75,0.70,0.68,0.66,0.42,0.38,0.41,0.07,0.05,0.05,0.04,0.04,0.04,0.82,0.70,0.63,0.59,0.56,0.28,0.27,0.27,0.06
75%,84345.00,0.08,0.07,0.07,0.07,0.07,0.89,0.83,0.79,0.77,0.75,0.59,0.55,0.59,0.09,0.07,0.06,0.06,0.06,0.06,0.89,0.80,0.75,0.71,0.69,0.43,0.42,0.44,0.07
max,93436.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.61,1.64,1.54,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.61,1.64,1.50,1.00


In [25]:
h1 = [3, 7, 11, 15, 19]
h2 = [0, 3, 7, 11, 15, 19]
percent_within_h_equ_w_mean = [f"percent_within_{i}_equ_w_mean" for i in h1]
percent_within_h_ow_weighted = [f"percent_within_{i}_ow_weighted" for i in h1]
percent_benchmark_h_equ_w_mean = [f"percent_benchmark_{i}_equ_w_mean" for i in h2]
percent_benchmark_h_ow_weighted = [f"percent_benchmark_{i}_ow_weighted" for i in h2]

In [26]:
two_weighted_average[percent_within_h_equ_w_mean].describe().round(4)

,percent_within_3_equ_w_mean,percent_within_7_equ_w_mean,percent_within_11_equ_w_mean,percent_within_15_equ_w_mean,percent_within_19_equ_w_mean
count,811361.00,807841.00,805660.00,805117.00,804979.00
mean,0.83,0.75,0.70,0.67,0.66
std,0.09,0.12,0.13,0.14,0.14
min,0.00,0.00,0.00,0.00,0.00
25%,0.78,0.67,0.61,0.58,0.56
50%,0.84,0.75,0.70,0.68,0.66
75%,0.89,0.83,0.79,0.77,0.75
max,1.00,1.00,1.00,1.00,1.00


In [27]:
two_weighted_average[percent_within_h_ow_weighted].describe().round(4)

,percent_within_3_ow_weighted,percent_within_7_ow_weighted,percent_within_11_ow_weighted,percent_within_15_ow_weighted,percent_within_19_ow_weighted
count,812346.00,812346.00,812346.00,812346.00,812346.00
mean,0.81,0.70,0.64,0.60,0.57
std,0.11,0.15,0.17,0.18,0.18
min,0.00,0.00,0.00,0.00,0.00
25%,0.75,0.61,0.53,0.48,0.46
50%,0.82,0.70,0.63,0.59,0.56
75%,0.89,0.80,0.75,0.71,0.69
max,1.00,1.00,1.00,1.00,1.00


In [28]:
two_weighted_average[percent_benchmark_h_equ_w_mean].describe().round(4)

,percent_benchmark_0_equ_w_mean,percent_benchmark_3_equ_w_mean,percent_benchmark_7_equ_w_mean,percent_benchmark_11_equ_w_mean,percent_benchmark_15_equ_w_mean,percent_benchmark_19_equ_w_mean
count,368416.00,368485.00,367602.00,367277.00,366739.00,366878.00
mean,0.08,0.07,0.06,0.06,0.06,0.06
std,0.08,0.07,0.06,0.06,0.06,0.06
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.05,0.04,0.04,0.04,0.04,0.04
50%,0.07,0.06,0.05,0.05,0.05,0.05
75%,0.09,0.08,0.07,0.07,0.07,0.07
max,1.00,1.00,1.00,1.00,1.00,1.00


In [29]:
two_weighted_average[percent_benchmark_h_ow_weighted].describe().round(4)

,percent_benchmark_0_ow_weighted,percent_benchmark_3_ow_weighted,percent_benchmark_7_ow_weighted,percent_benchmark_11_ow_weighted,percent_benchmark_15_ow_weighted,percent_benchmark_19_ow_weighted
count,368416.00,368485.00,367602.00,367277.00,366739.00,366878.00
mean,0.07,0.06,0.06,0.05,0.05,0.05
std,0.08,0.07,0.06,0.06,0.06,0.06
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.04,0.03,0.03,0.03,0.03,0.03
50%,0.06,0.05,0.05,0.04,0.04,0.04
75%,0.07,0.07,0.06,0.06,0.06,0.06
max,1.00,1.00,1.00,1.00,1.00,1.00


In [30]:
two_weighted_average[['percent_benchmark_median_passive_equ_w_mean','percent_benchmark_avg_passive_equ_w_mean','percent_benchmark_largest_passive_equ_w_mean']].describe().round(4)

,percent_benchmark_median_passive_equ_w_mean,percent_benchmark_avg_passive_equ_w_mean,percent_benchmark_largest_passive_equ_w_mean
count,362216.00,360859.00,360788.00
mean,0.45,0.42,0.44
std,0.23,0.22,0.23
min,0.00,0.00,0.00
25%,0.28,0.27,0.28
50%,0.42,0.38,0.41
75%,0.59,0.55,0.59
max,1.61,1.64,1.54


In [31]:
two_weighted_average[['percent_benchmark_median_passive_ow_weighted','percent_benchmark_avg_passive_ow_weighted','percent_benchmark_largest_passive_ow_weighted']].describe().round(4)

,percent_benchmark_median_passive_ow_weighted,percent_benchmark_avg_passive_ow_weighted,percent_benchmark_largest_passive_ow_weighted
count,362216.00,360859.00,360788.00
mean,0.34,0.33,0.34
std,0.23,0.22,0.23
min,0.00,0.00,0.00
25%,0.18,0.17,0.17
50%,0.28,0.27,0.27
75%,0.43,0.42,0.44
max,1.61,1.64,1.50


## Compute Coverage Ratio

In [32]:
market_cap = pd.read_stata('PERMNO_ME.dta')
# Convert 'yearm' to datetime format in 'market_cap'
market_cap['yearm'] = pd.to_datetime(market_cap['yearm'])

# Extract year and quarter from 'yearm' in 'market_cap'
market_cap['year'] = market_cap['yearm'].dt.year
market_cap['quarter'] = market_cap['yearm'].dt.quarter

# Combine year and quarter into a new column 'yearQx' in 'market_cap'
# Note: Adjusted column name to 'yearQx' to match the description, though the code combines them into 'quarter'
market_cap['quarter'] = market_cap['year'].astype(str) + 'Q' + market_cap['quarter'].astype(str)
market_cap = market_cap[['permno','quarter','mkt_cap']]
market_cap = market_cap.replace(0,np.nan)

In [33]:
measures = ['percent_within_3', 'percent_within_7','percent_within_11',
        'percent_within_15', 'percent_within_19', 'percent_benchmark_0',
       'percent_benchmark_3', 'percent_benchmark_7', 'percent_benchmark_11',
       'percent_benchmark_15', 'percent_benchmark_19',
       'percent_benchmark_median_passive', 'percent_benchmark_avg_passive',
       'percent_benchmark_largest_passive']

dfs = []

for measure in measures:
    # Filter and retain only necessary columns
    filtered_df = mutual_fund_holding[['permno', 'quarter', 'dollar_holdings', measure]]
    filtered_df = filtered_df[filtered_df[measure].notna()]
    
    # Group by 'permno' and 'quarter', summing the 'dollar_holdings'
    grouped_df = filtered_df.groupby(['permno', 'quarter'])['dollar_holdings'].sum().reset_index()
    
    # Merge with market_cap
    merged_df = pd.merge(grouped_df, market_cap, on=['permno', 'quarter'])
    
    # Calculate the coverage ratio
    merged_df[f'{measure}_coverage_ratio'] = merged_df['dollar_holdings'] / (merged_df['mkt_cap']*1000)
    
    # Retain only required columns
    merged_df = merged_df[['permno', 'quarter', f'{measure}_coverage_ratio']]
    
    # Append to list of DataFrames
    dfs.append(merged_df)

# Combine all DataFrames into one, merging on 'permno' and 'quarter'
coverage_ratio_df = dfs[0]
for df in dfs[1:]:
    coverage_ratio_df = pd.merge(coverage_ratio_df, df, on=['permno', 'quarter'], how='outer')
    

In [54]:
coverage_ratio_df

,permno,quarter,percent_within_3_coverage_ratio,percent_within_7_coverage_ratio,percent_within_11_coverage_ratio,percent_within_15_coverage_ratio,percent_within_19_coverage_ratio,percent_benchmark_0_coverage_ratio,percent_benchmark_3_coverage_ratio,percent_benchmark_7_coverage_ratio,percent_benchmark_11_coverage_ratio,percent_benchmark_15_coverage_ratio,percent_benchmark_19_coverage_ratio,percent_benchmark_median_passive_coverage_ratio,percent_benchmark_avg_passive_coverage_ratio,percent_benchmark_largest_passive_coverage_ratio
0,10001,1994Q3,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,1995Q2,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,1995Q3,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,1995Q4,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,1996Q1,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702550,93436,2019Q3,0.04,0.04,0.04,0.03,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
702551,93436,2019Q4,0.07,0.07,0.07,0.07,0.07,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04
702552,93436,2020Q1,0.05,0.04,0.04,0.04,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
702553,93436,2020Q2,0.08,0.08,0.08,0.08,0.08,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04


In [34]:
coverage_ratio_df.describe()

,permno,percent_within_3_coverage_ratio,percent_within_7_coverage_ratio,percent_within_11_coverage_ratio,percent_within_15_coverage_ratio,percent_within_19_coverage_ratio,percent_benchmark_0_coverage_ratio,percent_benchmark_3_coverage_ratio,percent_benchmark_7_coverage_ratio,percent_benchmark_11_coverage_ratio,percent_benchmark_15_coverage_ratio,percent_benchmark_19_coverage_ratio,percent_benchmark_median_passive_coverage_ratio,percent_benchmark_avg_passive_coverage_ratio,percent_benchmark_largest_passive_coverage_ratio
count,702555.00,702295.00,701229.00,699844.00,699668.00,699548.00,351094.00,351161.00,350313.00,350033.00,349769.00,349894.00,344903.00,343731.00,343443.00
mean,61231.23,2.09,2.07,2.06,2.05,2.04,0.85,0.85,0.85,0.85,0.84,0.84,0.91,0.89,0.90
std,27871.72,102.71,102.23,101.77,101.38,101.31,45.84,45.82,45.82,45.81,45.82,45.80,48.77,49.09,48.74
min,10001.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,37102.00,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
50%,75730.00,0.08,0.08,0.08,0.08,0.08,0.06,0.06,0.06,0.06,0.06,0.06,0.07,0.07,0.07
75%,84234.00,0.22,0.22,0.22,0.22,0.22,0.13,0.13,0.13,0.13,0.13,0.13,0.14,0.14,0.14
max,93436.00,25115.21,24842.48,24656.22,24656.22,24656.22,15540.58,15540.58,15540.58,15540.58,15540.58,15540.58,16102.87,16722.82,16137.05


In [56]:
merged_df = pd.merge(two_weighted_average,coverage_ratio_df,on=['permno','quarter'],how='left')

In [57]:
merged_df.describe()

,permno,percent_benchmark_3_equ_w_mean,percent_benchmark_7_equ_w_mean,percent_benchmark_11_equ_w_mean,percent_benchmark_15_equ_w_mean,percent_benchmark_19_equ_w_mean,percent_within_3_equ_w_mean,percent_within_7_equ_w_mean,percent_within_11_equ_w_mean,percent_within_15_equ_w_mean,percent_within_19_equ_w_mean,percent_benchmark_median_passive_equ_w_mean,percent_benchmark_avg_passive_equ_w_mean,percent_benchmark_largest_passive_equ_w_mean,percent_benchmark_0_equ_w_mean,percent_benchmark_3_ow_weighted,percent_benchmark_7_ow_weighted,percent_benchmark_11_ow_weighted,percent_benchmark_15_ow_weighted,percent_benchmark_19_ow_weighted,percent_within_3_ow_weighted,percent_within_7_ow_weighted,percent_within_11_ow_weighted,percent_within_15_ow_weighted,percent_within_19_ow_weighted,percent_benchmark_median_passive_ow_weighted,percent_benchmark_avg_passive_ow_weighted,percent_benchmark_largest_passive_ow_weighted,percent_benchmark_0_ow_weighted,percent_within_3_coverage_ratio,percent_within_7_coverage_ratio,percent_within_11_coverage_ratio,percent_within_15_coverage_ratio,percent_within_19_coverage_ratio,percent_benchmark_0_coverage_ratio,percent_benchmark_3_coverage_ratio,percent_benchmark_7_coverage_ratio,percent_benchmark_11_coverage_ratio,percent_benchmark_15_coverage_ratio,percent_benchmark_19_coverage_ratio,percent_benchmark_median_passive_coverage_ratio,percent_benchmark_avg_passive_coverage_ratio,percent_benchmark_largest_passive_coverage_ratio
count,812346.00,368485.00,367602.00,367277.00,366739.00,366878.00,811361.00,807841.00,805660.00,805117.00,804979.00,362216.00,360859.00,360788.00,368416.00,368485.00,367602.00,367277.00,366739.00,366878.00,812346.00,812346.00,812346.00,812346.00,812346.00,362216.00,360859.00,360788.00,368416.00,702295.00,701229.00,699844.00,699668.00,699548.00,351094.00,351161.00,350313.00,350033.00,349769.00,349894.00,344903.00,343731.00,343443.00
mean,61742.23,0.07,0.06,0.06,0.06,0.06,0.83,0.75,0.70,0.67,0.66,0.45,0.42,0.44,0.08,0.06,0.06,0.05,0.05,0.05,0.81,0.70,0.64,0.60,0.57,0.34,0.33,0.34,0.07,2.09,2.07,2.06,2.05,2.04,0.85,0.85,0.85,0.85,0.84,0.84,0.91,0.89,0.90
std,27777.95,0.07,0.06,0.06,0.06,0.06,0.09,0.12,0.13,0.14,0.14,0.22,0.22,0.23,0.08,0.07,0.06,0.06,0.06,0.06,0.11,0.15,0.17,0.18,0.18,0.23,0.22,0.23,0.08,102.71,102.23,101.77,101.38,101.31,45.84,45.82,45.82,45.81,45.82,45.80,48.77,49.09,48.74
min,10001.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,38295.00,0.04,0.04,0.04,0.04,0.04,0.78,0.67,0.61,0.58,0.56,0.28,0.27,0.28,0.05,0.03,0.03,0.03,0.03,0.03,0.75,0.61,0.53,0.48,0.46,0.18,0.17,0.17,0.04,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
50%,76023.00,0.06,0.05,0.05,0.05,0.05,0.84,0.75,0.70,0.68,0.66,0.42,0.38,0.41,0.07,0.05,0.05,0.04,0.04,0.04,0.82,0.70,0.63,0.59,0.56,0.28,0.27,0.27,0.06,0.08,0.08,0.08,0.08,0.08,0.06,0.06,0.06,0.06,0.06,0.06,0.07,0.07,0.07
75%,84345.00,0.08,0.07,0.07,0.07,0.07,0.89,0.83,0.79,0.77,0.75,0.59,0.55,0.59,0.09,0.07,0.06,0.06,0.06,0.06,0.89,0.80,0.75,0.71,0.69,0.43,0.42,0.44,0.07,0.22,0.22,0.22,0.22,0.22,0.13,0.13,0.13,0.13,0.13,0.13,0.14,0.14,0.14
max,93436.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.61,1.64,1.54,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.61,1.64,1.50,1.00,25115.21,24842.48,24656.22,24656.22,24656.22,15540.58,15540.58,15540.58,15540.58,15540.58,15540.58,16102.87,16722.82,16137.05


In [60]:
merged_df.to_csv('2024_04_20_stock_sparisty_coverage_ratio.csv',index=False)